In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import torch
import torch.utils.data as data 

from copy import deepcopy
from cv2 import aruco
from omegaconf import OmegaConf
from torch.nn.parallel import DistributedDataParallel as DDP
from tqdm import tqdm

# Custom imports
from contrastive_learning.datasets.state_dataset import StateDataset
from contrastive_learning.tests.plotting import plot_corners, plot_rvec_tvec


In [2]:
# Script to check the distribution of all the states in given dataset

In [3]:
# Set the needed variables for the mock dataset to retrieve data
cfg = OmegaConf.create()
cfg.data_dir = '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_1_demos/test_demos'
cfg.pos_ref = 'global' 
cfg.pos_type = 'corners'
cfg.pos_dim = 8

In [4]:
print(cfg)

{'data_dir': '/home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_1_demos/train_demos', 'pos_ref': 'global', 'pos_type': 'corners', 'pos_dim': 8}


In [5]:
dataset = StateDataset(cfg)
data_loader = data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4)

DATASET POS_REF: global
len(dataset): 4350
self.action_min: [-0.15000001 -0.30000001], self.action_max: [0.15000001 0.30000001]


In [6]:
# Method to dump all the positions to test_demos
def dump_all_pos(bs, dataset, data_loader):
    pbar = tqdm(total=len(data_loader))
    all_curr_pos = np.zeros((len(dataset), cfg.pos_dim*2))
    for i,batch in enumerate(data_loader):
        curr_pos, _, _ = [b for b in batch] # These are normalized
        all_curr_pos[i*bs:(i+1)*bs, :] = curr_pos.detach().numpy()
        pbar.update(1)

    with open(os.path.join(cfg.data_dir, 'all_curr_pos.npy'), 'wb') as f:
        np.save(f, all_curr_pos)
        
    print('All positions saved to : {}'.format(os.path.join(cfg.data_dir, 'all_curr_pos.npy')))

In [7]:
dump_all_pos(bs=1, dataset=dataset, data_loader=data_loader)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4350/4350 [00:03<00:00, 1347.24it/s]

All positions saved to : /home/irmak/Workspace/DAWGE/src/dawge_planner/data/box_orientation_1_demos/train_demos/all_curr_pos.npy


In [8]:
# Method to find the average MSE to the closest k neighbours of curr_pos to all_curr_pos.npy
def get_mse_to_dist(cfg, curr_pos, k=10):
    with open(os.path.join(cfg.data_dir, 'all_curr_pos.npy'), 'rb') as f:
        all_curr_pos = np.load(f)

    dist = np.linalg.norm(all_curr_pos - curr_pos, axis=1)
    dist.sort()

    return sum(dist[:k])


In [9]:
def get_mse_for_all(cfg):
    # It will traverse through the all_curr_pos.npy and find the mse distance for all demos
    with open(os.path.join(cfg.data_dir, 'all_curr_pos.npy'), 'rb') as f:
        all_curr_pos = np.load(f)
        
    print('all_curr_pos: {}'.format(all_curr_pos))
        
    all_mses = []
    for curr_pos in all_curr_pos:
        all_mses.append(get_mse_to_dist(cfg, curr_pos, k=10))
        
    return all_mses

In [10]:
all_mses = get_mse_for_all(cfg)

all_curr_pos: [[0.18485342 0.52867132 0.18648209 ... 0.83636361 0.08713355 0.84475523]
 [0.18485342 0.52867132 0.18648209 ... 0.82517481 0.08387622 0.83356643]
 [0.18485342 0.52867132 0.18648209 ... 0.79160839 0.08306189 0.79160839]
 ...
 [0.29234529 0.21958043 0.30700326 ... 0.30349651 0.25977197 0.29090908]
 [0.29885992 0.21398601 0.31351793 ... 0.29650348 0.26791531 0.28251749]
 [0.30537459 0.20839161 0.31921825 ... 0.2881119  0.27605864 0.27412587]]


In [11]:
plt.plot(all_mses)
plt.ylabel("Normalized Norm Dist")
plt.xlabel("Frame ID")
plt.savefig(os.path.join(cfg.data_dir, 'all_mses.png'))

In [ ]:
# Plot this value for all the demos